In [2]:
from decouple import config


In [3]:
assert config('POLOGYON_API_KEY', default=None, cast=str) is not None


In [4]:
import requests

In [27]:
ticker = 'AAPL'
multiplier = '5'
timespan = 'minute'
from_date = '2023-01-01'
to_date = '2023-12-31'


path = f'/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}'
url = f"https://api.polygon.io{path}?apikey={POLOGYON_API_KEY}"

In [33]:
import pytz
from datetime import datetime
from typing import Literal
from urllib.parse import urlencode
from dataclasses import dataclass

POLOGYON_API_KEY = config('POLOGYON_API_KEY', default=None, cast=str)


def transform_polygon_result(result):
    unix_timestampt = result.get('t') / 1000.0
    utc_timestampt = datetime.fromtimestamp(unix_timestampt, tz=pytz.timezone('UTC'))
    print(unix_timestampt)
    return {
        'open_price': result['o'],
        'close_price': result['c'],
        'high_price': result['h'],
        'low_price': result['l'],
        'number_of_trades': result['n'],
        'volume': result['v'],
        'volume_weighted_average': result['vw'],
        'time': utc_timestampt
    }


@dataclass
class PolygonAPIClient:
    ticker: str = 'AAPL'
    multiplier: int = 5
    timespan: str = 'minute'
    from_date: str = '2023-01-01'
    to_date: str = '2023-12-31'
    api_key: str = ''
    adjusted: bool = True
    sort: Literal['asc','desc'] = 'asc'
    
    def get_api_key(self):
        return self.api_key or POLOGYON_API_KEY
    
    def get_headers(self):
        api_key = self.get_api_key()
        return {
            "Authorization": f'Bearer {api_key}'
        }
        
    def get_params(self):
        return {
            'adjusted': self.adjusted,
            'sort': self.sort
        }
    
    def generate_url(self, pass_auth=False):
        path = f'/v2/aggs/ticker/{self.ticker}/range/{self.multiplier}/{self.timespan}/{self.from_date}/{self.to_date}'
        url = f"https://api.polygon.io{path}"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f'{url}?{encoded_params}'
        if pass_auth:
            api_key = self.get_api_key()
            url += f'&api_key={api_key}'
        return url
    
    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() # not 200/201
        return response.json()
    
    
    def get_stock_data(self):
        data = self.fetch_data()
        results = data['results']
        dataset = []
        for result in results:
            dataset.append(
                transform_polygon_result(result)
            )
        return dataset
        

In [32]:
stock_api_client = PolygonAPIClient(ticker='GOOG',multiplier=1)

stock_api_client.generate_url()

'https://api.polygon.io/v2/aggs/ticker/GOOG/range/1/minute/2023-01-01/2023-12-31?adjusted=True&sort=asc'

In [34]:
dataset = stock_api_client.get_stock_data()
dataset


1692086400.0
1692086460.0
1692086520.0
1692087660.0
1692087840.0
1692088260.0
1692088500.0
1692088560.0
1692089100.0
1692089160.0
1692090000.0
1692092820.0
1692093300.0
1692093420.0
1692093960.0
1692094200.0
1692094260.0
1692094680.0
1692094740.0
1692094980.0
1692095040.0
1692095100.0
1692095340.0
1692095400.0
1692095520.0
1692095580.0
1692096120.0
1692096180.0
1692097200.0
1692097260.0
1692097320.0
1692097440.0
1692097500.0
1692097800.0
1692098280.0
1692098640.0
1692098820.0
1692099540.0
1692099600.0
1692099660.0
1692100260.0
1692100500.0
1692100800.0
1692100860.0
1692100980.0
1692101100.0
1692101220.0
1692101340.0
1692101400.0
1692102120.0
1692102240.0
1692102300.0
1692102480.0
1692102600.0
1692102660.0
1692102720.0
1692102780.0
1692102840.0
1692102900.0
1692102960.0
1692103080.0
1692103140.0
1692103200.0
1692103260.0
1692103380.0
1692103440.0
1692103620.0
1692103920.0
1692103980.0
1692104040.0
1692104100.0
1692104160.0
1692104400.0
1692104520.0
1692104640.0
1692104700.0
1692104760.0

[{'open_price': 131.6,
  'close_price': 131.72,
  'high_price': 131.72,
  'low_price': 131,
  'number_of_trades': 41,
  'volume': 1960,
  'volume_weighted_average': 131.3027,
  'time': datetime.datetime(2023, 8, 15, 8, 0, tzinfo=<UTC>)},
 {'open_price': 131.27,
  'close_price': 131.27,
  'high_price': 131.27,
  'low_price': 131.27,
  'number_of_trades': 5,
  'volume': 262,
  'volume_weighted_average': 131.2738,
  'time': datetime.datetime(2023, 8, 15, 8, 1, tzinfo=<UTC>)},
 {'open_price': 131.27,
  'close_price': 131.27,
  'high_price': 131.27,
  'low_price': 131.27,
  'number_of_trades': 1,
  'volume': 100,
  'volume_weighted_average': 131.27,
  'time': datetime.datetime(2023, 8, 15, 8, 2, tzinfo=<UTC>)},
 {'open_price': 131.15,
  'close_price': 131.15,
  'high_price': 131.15,
  'low_price': 131.15,
  'number_of_trades': 3,
  'volume': 160,
  'volume_weighted_average': 131.1462,
  'time': datetime.datetime(2023, 8, 15, 8, 21, tzinfo=<UTC>)},
 {'open_price': 131.14,
  'close_price': 13